In [ ]:
from rtlsdr import RtlSdr
import pylab


%matplotlib widget

In [ ]:
# NOTE(dominic): This is super lame, but iptyhon kernels are running in the directory they are
# contained. That means, we cannot access any other sub-modules from within these notebooks.
# We resolve this by temporarily adding the parent directory to the path.

import os
import sys
sys.path.insert(0, os.path.abspath('..'))


from util.plotting import compute_fft_plot, compute_fft_plot_from_sample_rate, compute_frequency_response
from util.filtering import compute_lpf_coeff, filter_complex_signal, filter_real_signal
from util.data_io import read_sdriq_data
from util.demodulation import demodulate_signal, chunked_demodulate_signal

In [ ]:
import numpy as np
from scipy.io import wavfile

In [ ]:
fs = 2048000
center_frequency = 98500000

signal = read_sdriq_data("/home/dominic/radio/long_test.sdriq")

In [ ]:
pylab.figure(figsize=(5,5))

pylab.psd(signal, NFFT=1024, Fs=fs, Fc=center_frequency)
pylab.xlabel('Frequency (MHz)')
pylab.ylabel('Relative power (dB)')

pylab.show()

In [ ]:
pylab.figure(figsize=(5,5))

freq, mag = compute_fft_plot_from_sample_rate(signal, sampling_rate=fs, center_frequency=98500000)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
sos = compute_lpf_coeff(cutoff_frequency=75000, sample_rate=fs)

frequencies, magnitudes = compute_frequency_response(sos, fs)

pylab.figure(figsize=(5,5))
pylab.plot(frequencies, magnitudes)
pylab.show()

In [ ]:
filtered_signal = filter_complex_signal(signal=signal, cutoff_frequency=75000, sample_rate=fs)

In [ ]:
pylab.figure(figsize=(5,5))

pylab.psd(filtered_signal, NFFT=1024, Fs=fs, Fc=center_frequency)
pylab.xlabel('Frequency (MHz)')
pylab.ylabel('Relative power (dB)')

pylab.show()

In [ ]:
pylab.figure(figsize=(5,5))

freq, mag = compute_fft_plot_from_sample_rate(filtered_signal, sampling_rate=fs, center_frequency=center_frequency)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
angles = np.angle(filtered_signal)
# Difference the phase angle and convert to +/-pi range.
angle_diff = np.unwrap(np.diff(angles))

In [ ]:
pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(angle_diff, sampling_rate=fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
filtered_angle_diff = filter_real_signal(signal=angle_diff, cutoff_frequency=20e3, sample_rate=fs)


pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(filtered_angle_diff, sampling_rate=fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
angle_diff_downsampled = filtered_angle_diff[::25]
angle_diff_downsampled_fs = fs/25

pylab.figure()

freq, mag = compute_fft_plot_from_sample_rate(angle_diff_downsampled, sampling_rate=angle_diff_downsampled_fs, center_frequency=0)

pylab.plot(freq, mag)

pylab.show()

In [ ]:
wavfile.write('/home/dominic/radio/data/test.wav', int(angle_diff_downsampled_fs), angle_diff_downsampled)

In [ ]:
fs = 2048000
center_frequency = 98500000

signal = read_sdriq_data("/home/dominic/radio/long_test.sdriq")

audio_signal, audio_signal_fs = chunked_demodulate_signal(signal, sample_rate=fs)
wavfile.write('/home/dominic/radio/data/test.wav', int(audio_signal_fs), audio_signal)